In [1]:
# Setup: Imports

import requests
import pandas as pd
import numpy as np
import json
from datetime import datetime, timedelta
import os
import asyncio
import time
import aiohttp
import async_timeout

from asyncio import _get_running_loop as _get_running_loop

In [2]:
# Progress Bar


def progress_bar(progress, total, obj):
    percent = 100*(float(progress)/float(total))
    bar = '█'*int(percent)+'-'*(100-int(percent))
    print(f"\r|{bar}| {percent: .2f}% Processing : {obj}", end='\r')

In [3]:
# Setup: Get API Keys and Connection.


def read_creds(cred_loc, main=1, backup=0):
    if os.path.exists(cred_loc):
        f = open(cred_loc)
        creds = json.load(f)
        f.close()

        api_key = creds["nagmani.b@exalogic.co"]

        emp_url = creds["emp_url"]
        app_url = creds["app_url"]
        print("Received the Credentials.")
        return api_key, emp_url, app_url
    else:
        print("File not found")
        return []

In [4]:
# Extract: Extract the Data from Desktime app.


def get_dt_data(url: str, api: str, dt: str, emp=""):
    if emp:
        try:
            r = requests.get(
                url, params={"apiKey": api, "date": dt, "id": emp})
        except Exception as e:
            print(e)
            return ""
        else:
            # print("Desktime Connection Successful.")
            return r.json()
    else:
        try:
            # print("Desktime Connection Successful.")
            r = requests.get(url, params={"apiKey": api, "date": dt})
        except Exception as e:
            print(e)
            return ""
        else:
            return r.json()

In [5]:
# Transform: Final DT API Processing Function.


def process_dt_api(dt, resp, df_act_emp):
    emp_logged = []
    today = resp['employees'][dt]
    for emp in today:
        emp_logged.append(emp)

    remove_cols = ['profileUrl', 'activeProject', 'notes', 'afterWorkTime', 'beforeWorkTime',
                   'work_starts', 'work_ends', 'arrived', 'left', 'late', 'isOnline']
    cols = []

    for x in resp['employees'][dt][emp_logged[0]]:
        if not remove_cols.__contains__(x):
            cols.append(x)
    dict1 = {}
    for col in cols:
        dict1[col] = []

    for emp in emp_logged:
        tmp = resp['employees'][dt][emp]
        dict1['id'].append(str(tmp['id']))
        dict1['name'].append(tmp['name'])
        dict1['email'].append(tmp['email'])
        dict1['groupId'].append(tmp["groupId"])
        dict1['group'].append(tmp["group"])
        dict1['onlineTime'].append(tmp["onlineTime"])
        dict1['offlineTime'].append(tmp["offlineTime"])
        dict1['desktimeTime'].append(tmp["desktimeTime"])
        dict1['atWorkTime'].append(tmp["atWorkTime"])
        dict1['productiveTime'].append(tmp["productiveTime"])
        dict1['productivity'].append(tmp["productivity"])
        dict1['efficiency'].append(tmp["efficiency"])

    df1 = pd.DataFrame(dict1)
    df1.reset_index(inplace=True, drop=True)

    df_merged = df_act_emp.merge(df1,
                                 left_on="Email",
                                 right_on="email",
                                 suffixes=("_left", "_right"),
                                 how="left")
    df_merged["date"] = dt
    df_merged["count"] = 1
    print("Complete Data converted to a Dataframe.")
    return df_merged


def update_exist_emp(exist_emp_df, df_dt_usage):
    df_tmp = df_dt_usage[["Emp ID", "Reporting Manager",
                          "Department", "Division", "id"]]
    df_merged = exist_emp_df.merge(
        df_tmp,
        left_on="Emp ID",
        right_on="Emp ID",
        how="left",
        suffixes=("_existing", "_new")
    )

    return df_merged

In [6]:
# Transform2: Get App Usage from Desktime per employee.


df_app_list = pd.DataFrame(columns=[
                           'Date', 'Name', 'App_Name', 'App_Type', 'App_Tag', 'Time_Mins', 'Category', 'Productive'])


def get_meeting_time(x):
    if x.__contains__('teams') | x.__contains__('CALENDAR') | x.__contains__('Teams'):
        return "Teams"
    else:
        return "Work"


def get_app_usage(emp_id, emp_resp, dt):
    dict_usage = {"Date": [], "Desktime_ID": [], "emp_app_name": [], "emp_time_mins": [
    ], "emp_time_secs": [], "emp_category": [], "emp_productive": []}
    # print(emp_resp)
    app_usage = emp_resp["apps"]['1']

    for app in app_usage:
        t_spent_mins = np.round(
            int(emp_resp["apps"]["1"][app]["duration"])/60, 2)
        t_spent_secs = int(emp_resp["apps"]["1"][app]["duration"])
        app_name = emp_resp["apps"]["1"][app]["app"]
        app_type = emp_resp["apps"]["1"][app]["type"]
        dict_usage["Date"].append(dt)
        dict_usage["Desktime_ID"].append(emp_id)
        dict_usage["emp_app_name"].append(app_name)
        dict_usage["emp_time_mins"].append(t_spent_mins)
        dict_usage["emp_time_secs"].append(t_spent_secs)
        dict_usage["emp_category"].append(get_meeting_time(app_name))
        dict_usage["emp_productive"].append("Productive")

    return pd.DataFrame(dict_usage)

## New Functions 

In [7]:
loop = asyncio.get_event_loop()

In [8]:
strt_dt = input("Enter Start Date in YYYY-MM-DD format :")

In [9]:
# Get all Employee Details
# ===========================================================================================

async def dt_fetch_all(session, url, api, dt,emp):
  async with async_timeout.timeout(15):
    async with session.get(url,params={"apiKey": api, "date": dt}) as response:
      return await response.json()
  
async def dt_fetch_all(loop,api,dt,emp, *urls):
  tasks = []
  async with aiohttp.ClientSession(loop=loop) as session:
    for url in urls:
      tasks.append(dt_fetch_all(session, url, api,dt,emp))
    grouped_tasks = asyncio.gather(*tasks)
    return await grouped_tasks


In [10]:
# Get Individual Employees' Apps
# ===========================================================================================

async def dt_fetch_apps(session, url, api, dt,emp):
  async with async_timeout.timeout(15):
    async with session.get(url,params={"apiKey": api, "date": dt,"id":emp}) as response:
      return await response.json()
  
async def dt_fetch_emps(loop,api,dt,emp, *urls):
  tasks = []
  async with aiohttp.ClientSession(loop=loop) as session:
    for url in urls:
      tasks.append(dt_fetch_apps(session, url, api,dt,emp))
    grouped_tasks = asyncio.gather(*tasks)
    return await grouped_tasks


In [11]:
loc = "C:/Users/ShashankRaj/OneDrive - Exalogic Consulting/Documents/Proj Mgmt/Jira_Clockwork"
urls = []
cred_file = loc+"/Creds/dt_cred.json"
date_format = '%Y-%m-%d'
n=1

# Get the date range for which this code will execute
date_range = []
if len(strt_dt) < 2:
  curr_dt = str(datetime.today()-timedelta(days=int(0)+1)).split(" ")[0]
  date_range.append(curr_dt)
  print(f'Start date is {curr_dt}')
else:
    for dt in np.arange(n):
      curr_dt = str(datetime.strptime(strt_dt, date_format) -
                      timedelta(days=int(dt)+1)).split(" ")[0]
      date_range.append(curr_dt)


# Read the credentials
api_key, all_emps, app_url = read_creds(cred_file, main=1)
# dt_emp_dump = loop.run_until.complete(dt_fetch_all(loop,api_key,dt,all_emps, *urls))
# _get_running_loop

dt_emp_dump = loop.run_until_complete(dt_fetch_all(loop,api_key,dt,all_emps, *urls))


Received the Credentials.


RuntimeError: This event loop is already running